In [ ]:
%matplotlib inline
# import
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import json
import os
import pathlib
from matplotlib.colors import ListedColormap

net_birth = "../../../eval-results/_server-results/net-BP-birth/net.eval" 
net_uni = "../../../eval-results/_server-results/net-BP-uni/net.eval" 
net_sap = "../../../eval-results/_server-results/net-BP-sap/net.eval" 

bp_birth = "../../../eval-results/_server-results/test13_ppp/"
bp_uni = "../../../eval-results/_server-results/test17_ppp/"
bp_sap = "../../../eval-results/_server-results/test18_ppp/"

pre_birth = "../../../eval-results/_server-results/test12_ppp/"
pre_uni = "../../../eval-results/_server-results/test15_ppp/"
pre_sap = "../../../eval-results/_server-results/test16_ppp/"


In [ ]:
def net1adder(row):
    #get net names
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['Name'].split('-')[0].replace(".pnml","")
    else:
        return 0
def net2adder(row):        
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['Name'].split('-')[1].split('.')[0].replace(".pnml","")
    else: 
        return 0
    
def pnml_remover(row):
    #get net names
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['Name'].replace(".pnml","").replace(".rdf","")
    else:
        return row['Name']

def net1NonTaus(row):
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return df_nets.at[row['net1'],'nNonSilentTransitions'] 
    else:
        return 0
def net2NonTaus(row):
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return df_nets.at[row['net2'],'nNonSilentTransitions'] 
    else:
        return 0
def sumNonTaus(row):    
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['net1NonTaus'] + row['net2NonTaus']
    else:
        return 0

def fetch_data(folder, net_analysis, dataset, profile):
    dir_list = next(os.walk(folder))[1]
    dfs = []
    #merge all 
    for subfolder in dir_list:
        #print(subfolder)
        evalFile = folder +"/" + subfolder +"/aggRetrospectiveResults.eval"
        confFile = folder +"/" + subfolder +"/config.log"
        if os.path.exists(evalFile):
            df = pd.read_csv(evalFile ,encoding="ISO-8859-1", skipinitialspace=True)
            with open(confFile) as json_file:
                conf = json.load(json_file)
                #add config information to dataframe 
                df['matcher'] = conf['matcher']['ilp'] + " - " + conf['matcher']['profile'] +" - " + str(conf['matcher']['word-sim']) + " - sim-weight=" + str(conf['matcher']['sim-weight']) + " - match-postprocessing=" + str(conf['matcher']['postprocessing-thresh'] ) + " - Node Limit: " +str(conf['matcher']['ilp-node-limit']) + "- Time Limit : " +str(conf['matcher']['ilp-time-limit'])
                df['matcher_wo_weight'] = conf['matcher']['ilp'] + " - " + conf['matcher']['profile'] +" - " + str(conf['matcher']['word-sim']) + " - match-postprocessing=" + str(conf['matcher']['postprocessing-thresh']) + " - Node Limit: " +str(conf['matcher']['ilp-node-limit']) + "- Time Limit : " +str(conf['matcher']['ilp-time-limit'])
                df['matcher_wo_limit'] = conf['matcher']['ilp'] + " - " + conf['matcher']['profile'] +" - " + str(conf['matcher']['word-sim']) + " - sim-weight=" + str(conf['matcher']['sim-weight']) + " - match-postprocessing=" + str(conf['matcher']['postprocessing-thresh'] )
                df['complex-matches'] = conf['matcher']['complex matches']
                df['profile'] = conf['matcher']['profile']
                df['ilp'] =  conf['matcher']['ilp']
                df['word-sim'] =  conf['matcher']['word-sim']
                df['sim-weight'] = conf['matcher']['sim-weight']
                df['matcher-postprocessing-threshold'] = conf['matcher']['postprocessing-thresh']
                df['eval-postprocessing-threshold'] = conf['evaluation']['postprocessing-thresh']
                df['ILP-time-limit'] = conf['matcher']['ilp-time-limit']
                df['ILP-node-limit'] = conf['matcher']['ilp-node-limit']
            
            #df.set_index(['Name','matcher'])
            dfs.append(df)

    df_combined = pd.concat(dfs)

    #convert time
    df_combined['OVERALL TIME'] = df_combined['OVERALL TIME'].map(lambda x: x / 1000000000.)
    df_combined['BP TIME'] = df_combined['BP TIME'].map(lambda x: x / 1000000000.)
    df_combined['LABEL-SIM TIME'] = df_combined['LABEL-SIM TIME'].map(lambda x: x / 1000000000.)
    df_combined['LP TIME'] = df_combined['LP TIME'].map(lambda x: x / 1000000000.)
    df_combined['Dataset'] = dataset
    df_combined['Prematching'] = profile

    #extend with net information stored in net_analysis
    df_nets = pd.read_csv(net_analysis ,encoding="ISO-8859-1", skipinitialspace=True)
    #df_nets.set_index('Name')


    df_combined['net1'] = df_combined.apply(lambda row: net1adder(row), axis=1)
    df_combined['net2'] = df_combined.apply(lambda row: net2adder(row), axis=1)
    #df_combined['net1NonTaus'] = df_combined.apply(lambda row: net1NonTaus(row), axis=1)
    #df_combined['net2NonTaus'] = df_combined.apply(lambda row: net2NonTaus(row), axis=1)
    #df_combined['sumNonTaus'] = df_combined.apply(lambda row: sumNonTaus(row), axis=1)
    df_combined['Name'] = df_combined.apply(lambda row: pnml_remover(row), axis=1)
    return df_combined

In [ ]:
# load 
bp = fetch_data(bp_birth, net_birth ,"Birth", "No")
bp = bp.append(fetch_data(bp_uni, net_uni, "Uni", "No"))
bp = bp.append(fetch_data(bp_sap, net_sap, "Sap", "No"))

pre = fetch_data(pre_birth, net_birth ,"Birth", "Yes")
pre = pre.append(fetch_data(pre_uni, net_uni, "Uni", "Yes"))
pre = pre.append(fetch_data(pre_sap, net_sap, "Sap", "Yes"))


# concat all three datasets
df_all = pd.concat([bp,pre])

#no postprocessing
df_all = df_all[df_all['eval-postprocessing-threshold'] == 0.0]
df_behavior_only = df_all[df_all['sim-weight'] == 1.0]
df_all = df_all[df_all['sim-weight'] < 1.0]

df_macro_avg = df_all[df_all.Name == "Aggregated (MACRO)"]
df_micro_avg = df_all[df_all.Name == "Aggregated (MICRO)"]

df_beh_only_macro = df_behavior_only[df_behavior_only.Name == "Aggregated (MACRO)"]
df_beh_only_micro = df_behavior_only[df_behavior_only.Name == "Aggregated (MICRO)"]

#write to excel
with pd.ExcelWriter('prematching-eval.xlsx') as writer:  
    df_macro_avg.to_excel(writer, sheet_name='Macro')
    df_micro_avg.to_excel(writer, sheet_name='Micro')
    df_all.to_excel(writer, sheet_name='All')
    
#select max fscore for each profile on each dataset
df_macro_max_fscore = df_macro_avg.groupby(['Dataset','Prematching']).max()["FSCORE"]
df_macro_max_fscore = df_macro_max_fscore.reset_index()

df_micro_max_fscore = df_micro_avg.groupby(['Dataset','Prematching']).max()["FSCORE"]
df_micro_max_fscore = df_micro_max_fscore.reset_index()

#select max fscore for each profile on each dataset
df_macro_max_fscore = df_macro_avg.groupby(['Dataset','Prematching']).max()["FSCORE"]
df_macro_max_fscore = df_macro_max_fscore.reset_index()

df_micro_max_fscore = df_micro_avg.groupby(['Dataset','Prematching']).max()["FSCORE"]
df_micro_max_fscore = df_micro_max_fscore.reset_index()

#select max fscore for each profile on each dataset
df_macro_min_fscore = df_macro_avg.groupby(['Dataset','Prematching']).min()["FSCORE"]
df_macro_min_fscore = df_macro_min_fscore.reset_index()

df_micro_min_fscore = df_micro_avg.groupby(['Dataset','Prematching']).min()["FSCORE"]
df_micro_min_fscore = df_micro_min_fscore.reset_index()

print(df_macro_max_fscore)
plt.close('all') 

fig, (ax1, ax2) = plt.subplots(1,2)
fig.suptitle("Maximum Fscore, varying Sim-Weight and Post-processing Threshold")
fig.set_size_inches(18.5, 5.25)
sns.catplot(x="Dataset", y="FSCORE", hue="Prematching", data=df_macro_avg, kind="bar",ci=100, n_boot=0, estimator=np.mean, height=4, aspect=2, ax = ax1, palette=sns.color_palette("Reds_d", 4), order=["Birth","Uni","Sap"], hue_order =["No","Yes"])
#sns.catplot(x="Dataset", y="FSCORE", hue="Profile", legend=False, data=df_macro_avg, kind="swarm", dodge=0.6,s=15, height=4,marker=11, aspect=2,palette=["#000000","#000000","#000000","#000000"], ax = ax1, order=["Birth","Uni","Sap"], hue_order =["Label only","Alpha","BP","BP+"],  label='_nolegend_')
sns.catplot(x="Dataset", y="FSCORE", hue="Prematching", legend=False, data=df_macro_max_fscore, kind="swarm", dodge=0.6,s=15, height=4,marker=11, aspect=2,palette=["#000000","#000000"], ax = ax1, order=["Birth","Uni","Sap"], hue_order =["No","Yes"],  label='_nolegend_')
sns.catplot(x="Dataset", y="FSCORE", hue="Prematching", legend=False, data=df_macro_min_fscore, kind="swarm", dodge=0.6,s=15, height=4,marker=10, aspect=2, ax = ax1, palette=["#000000","#000000"], order=["Birth","Uni","Sap"], hue_order =["No","Yes"], label='_nolegend_')
sns.catplot(x="Dataset", y="FSCORE", hue="Prematching", legend=False, data=df_beh_only_macro, kind="swarm", dodge=0.6,s=5, height=4,marker="o", aspect=2, ax = ax1, palette=["#000000","#000000"], order=["Birth","Uni","Sap"], hue_order =["No","Yes"], label='_nolegend_')
ax1.set_ylim([0.0,0.65])
ax1.set_ylabel("Fscore")
ax1.set_title("Macro")
handles, labels = ax1.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax1.legend(by_label.values(), by_label.keys(),title="Prematching")

sns.catplot(x="Dataset", y="FSCORE", hue="Prematching", data=df_micro_avg, kind="bar",ci=100, n_boot=0, estimator=np.mean, height=4, aspect=2, ax = ax2, palette=sns.color_palette("Reds_d", 4), order=["Birth","Uni","Sap"], hue_order =["No","Yes"])
#sns.catplot(x="Dataset", y="FSCORE", hue="Profile", legend=False, data=df_macro_avg, kind="swarm", dodge=0.6,s=15, height=4,marker=11, aspect=2,palette=["#000000","#000000","#000000","#000000"], ax = ax1, order=["Birth","Uni","Sap"], hue_order =["Label only","Alpha","BP","BP+"],  label='_nolegend_')
sns.catplot(x="Dataset", y="FSCORE", hue="Prematching", legend=False, data=df_micro_max_fscore, kind="swarm", dodge=0.6,s=15, height=4,marker=11, aspect=2,palette=["#000000","#000000"], ax = ax2, order=["Birth","Uni","Sap"], hue_order =["No","Yes"],  label='_nolegend_')
sns.catplot(x="Dataset", y="FSCORE", hue="Prematching", legend=False, data=df_micro_min_fscore, kind="swarm", dodge=0.6,s=15, height=4,marker=10, aspect=2, ax = ax2, palette=["#000000","#000000"], order=["Birth","Uni","Sap"], hue_order =["No","Yes"], label='_nolegend_')
sns.catplot(x="Dataset", y="FSCORE", hue="Prematching", legend=False, data=df_beh_only_micro, kind="swarm", dodge=0.6,s=5, height=4,marker="o", aspect=2, ax = ax2, palette=["#000000","#000000"], order=["Birth","Uni","Sap"], hue_order =["No","Yes"], label='_nolegend_')
ax2.set_ylim([0.0,0.65])
ax2.set_ylabel("Fscore")
ax2.set_title("Micro")
handles, labels = ax2.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax2.legend(by_label.values(), by_label.keys(), title="Prematching")
